In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import optim

In [2]:
import sys
sys.path.insert(1, "../")

In [3]:
from train import *
from data_preprocessing import *
from Models.yolov8cls_path import *

In [4]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu') 

In [5]:
model = Model(num_classes=10)

In [6]:
data_path = '../../dummy_datasets/'
norms_path = os.path.join(data_path, 'norms.json')

In [7]:
means = get_means(path=norms_path, train_loader=None)
stds = get_stds(path=norms_path, train_loader=None)

Means are: [0.4405549168586731, 0.4407285749912262, 0.4381718039512634]
stds are: [0.25142669677734375, 0.25270089507102966, 0.25131651759147644]


In [8]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((224, 224)),
                                      transforms.Normalize(mean=means, std=stds)])

In [9]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)
val_dataset = ImageNetSubset(path=data_path, train=False, transform=transformations, half=False, show=False)

In [10]:
epochs=3
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=4)

In [11]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)

In [12]:
loss_fn = nn.NLLLoss()

In [13]:
prof = torch.profiler.profile(
        # schedule=torch.profiler.schedule(wait=0, warmup=0, active=1, repeat=1),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/darknet19'),
        record_shapes=True,
        profile_memory=True,
        with_flops=True,
        with_modules=True,
        with_stack=True)


Profiling your personal module 
https://pytorch.org/tutorials/beginner/profiler.html

In [14]:
history, gradient_stats, prof = train(epochs, train_loader, val_loader, model, optimizer, loss_fn, prof)

2024-11-16 18:25:15.257475 Epoch 1: 
2024-11-16 18:25:21.629539 Batch 1: 
torch.Size([30, 10, 1, 1])
torch.Size([30, 10])
[Train] Accuracy: 13.3333, Loss per batch: 2.2849
2024-11-16 18:25:33.310796 Batch 1: 
torch.Size([256, 10, 1, 1])
torch.Size([256, 10])
2024-11-16 18:25:44.577420 Batch 2: 
torch.Size([244, 10, 1, 1])
torch.Size([244, 10])
[Val] Accuracy: 10.0%, loss per batch: 2.3026
2024-11-16 18:25:55.961150 Epoch 2: 
2024-11-16 18:26:04.550771 Batch 1: 
torch.Size([30, 10, 1, 1])
torch.Size([30, 10])
[Train] Accuracy: 23.3333, Loss per batch: 2.2079
2024-11-16 18:26:16.975491 Batch 1: 
torch.Size([256, 10, 1, 1])
torch.Size([256, 10])
2024-11-16 18:26:26.980195 Batch 2: 
torch.Size([244, 10, 1, 1])
torch.Size([244, 10])
[Val] Accuracy: 10.0%, loss per batch: 2.3027
2024-11-16 18:26:39.092225 Epoch 3: 
2024-11-16 18:26:45.962882 Batch 1: 
torch.Size([30, 10, 1, 1])
torch.Size([30, 10])
[Train] Accuracy: 40.0, Loss per batch: 2.0945
2024-11-16 18:26:59.123262 Batch 1: 
torch.Size

## Inference time

In [58]:
from ultralytics import YOLO

In [59]:
model = YOLO('../Models/yolov8m-cls.pt')

In [60]:
# model = model.model

In [70]:
img = torch.rand(1,3,640,640)
_ = transforms.Compose([transforms.Normalize(mean=[0.5, 0.5, 0.5],
                        std=[0.5, 0.5, 0.5])])
img = _(img)

In [71]:
with torch.autograd.profiler.profile(use_device='cpu',
                                    record_shapes=True,
                                    # profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=True) as prof:
   model.predict(img)
print(prof.key_averages().table(row_limit=-1))


0: 640x640 chain 0.14, velvet 0.08, honeycomb 0.06, wall_clock 0.06, spotlight 0.05, 292.6ms
Speed: 8.0ms preprocess, 292.6ms inference, 9.6ms postprocess per image at shape (1, 3, 640, 640)
-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                         Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  Total MFLOPs  
-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  aten::empty         0.18%     529.000us         0.18%     529.000us       6.531us            81            --  
                     aten::to         0.01%      41.800us         0.24%     707.100us      64.282us            11            --  
             aten::lift_fresh         0.00%       2.500us         0.00%       2.500us       1.250us             2            --  
                aten::detach

In [65]:
6.451+41631.744+2.560 +1.229

41641.984

YOLOv8n</br>
Mine: 3.290265 GFLOPs ; On the website: 4.3 GFLOPs</br>
YOLOv8s</br>
Mine: 12.449996 GFLOPs ; On the website: 13.5 GFLOPs</br>
YOLOv8m</br>
Mine: 41.640755 GFLOPs ; On the website: 42.7 GFLOPs